In [1]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize, linewidth=200)

from scipy.stats import multinomial

from pprint import pprint
from tqdm import tqdm

In [2]:
D = 100  # 文書数
V = 10   # 語彙数

true_K = 3 # トピック数

In [3]:
# トピック分布のハイパーパラメータを設定
true_alpha = 1
true_alpha_k = [true_alpha] * true_K
#rng = np.random.default_rng()
#true_alpha_k = rng.uniform(low=1, high=2, size=true_K)

# トピック分布のパラメータを生成
true_theta_dk = np.random.dirichlet(alpha=true_alpha_k, size=D)

print("真の文書トピック分布 (D, K):")
pprint(true_theta_dk)

assert np.all(np.abs(true_theta_dk.sum(axis=1) - 1.0) < 1e-5)

真の文書トピック分布 (D, K):
array([[0.13468032, 0.52013309, 0.34518659],
       [0.54050726, 0.11091588, 0.34857686],
       [0.15055127, 0.38447929, 0.46496944],
       [0.18148145, 0.4410735 , 0.37744506],
       [0.60142306, 0.22724276, 0.17133419],
       [0.07875756, 0.40631677, 0.51492566],
       [0.2184729 , 0.11849074, 0.66303636],
       [0.3162288 , 0.66699309, 0.01677811],
       [0.06500208, 0.72837057, 0.20662735],
       [0.48520904, 0.45570968, 0.05908128],
       [0.25969208, 0.72350569, 0.01680223],
       [0.08787642, 0.78595216, 0.12617142],
       [0.12917201, 0.80180245, 0.06902554],
       [0.68004479, 0.07516566, 0.24478954],
       [0.21220881, 0.04143511, 0.74635608],
       [0.52364026, 0.201428  , 0.27493174],
       [0.78947818, 0.01602733, 0.19449449],
       [0.47362973, 0.4202298 , 0.10614047],
       [0.58625442, 0.21186531, 0.20188027],
       [0.35996232, 0.3483716 , 0.29166608],
       [0.35636867, 0.12515411, 0.51847723],
       [0.3286307 , 0.57580164, 0.09

In [4]:
# 単語分布のハイパーパラメータを設定
true_beta = 1
true_beta_v = [true_beta] * V
#rng = np.random.default_rng()
#true_beta_v = rng.uniform(low=1, high=2, size=V)

# 単語分布のパラメータを生成
true_phi_kv = np.random.dirichlet(alpha=true_beta_v, size=true_K)

print("真のトピック単語分布 (K, V):")
pprint(true_phi_kv)

assert np.all(np.abs(true_phi_kv.sum(axis=1) - 1.0) < 1e-5)

真のトピック単語分布 (K, V):
array([[0.01919865, 0.16633477, 0.05738285, 0.13268342, 0.07506414, 0.11336259, 0.09554946, 0.08638594, 0.07115881, 0.18287938],
       [0.32409829, 0.00954788, 0.01414065, 0.14323502, 0.01233908, 0.14850979, 0.01652362, 0.08823999, 0.0751005 , 0.16826518],
       [0.03943344, 0.19671669, 0.16368696, 0.09395512, 0.00485964, 0.06389049, 0.05565399, 0.0490521 , 0.08162794, 0.25112364]])


In [5]:
## テスト文書を生成

W = [] # 文書集合を初期化
Z = [] # トピック集合を初期化
N_d = [None] * D        # 各文書の単語数を初期化
N_dw = np.zeros((D, V)) # 文書ごとの各語彙の出現頻度を初期化

min_N_d = 100 # 各文書の単語数の上限
max_N_d = 200 # 各文書の単語数の下限

for d in tqdm(range(D)):
    # 単語数を生成
    N_d[d] = np.random.randint(low=min_N_d, high=max_N_d)
    # 各単語のトピックを初期化
    true_z_dn = [None] * N_d[d]
    # 各単語の語彙を初期化
    w_dn = [None] * N_d[d]

    for n in range(N_d[d]):
        # トピックを生成
        k = np.random.choice(true_K, p=true_theta_dk[d])
        true_z_dn[n] = k
        # 語彙を生成
        w = np.random.choice(V, p=true_phi_kv[k])
        w_dn[n] = w
        # 頻度をカウント
        N_dw[d, w] += 1

    # トピック集合を格納
    Z.append(true_z_dn)
    W.append(w_dn)

    #print(f"#{d}")
    #print(f"Number of words: {N_d[d]}")
    #print("Topics: ", end="")
    #pprint(true_z_dn)
    #print("Words: ", end="")
    #pprint(w_dn)

100%|██████████| 100/100 [00:00<00:00, 172.71it/s]


In [6]:
# テスト文書をファイルに出力
with open("lda.test.txt", mode="w") as f:
    print("\n".join([" ".join([str(w) for w in words]) for words in W]), file=f)

!head -n2 lda.test.txt

9 6 7 5 2 5 5 0 0 8 8 9 6 1 3 1 2 3 8 0 9 0 1 5 8 3 7 1 7 0 9 0 9 8 7 9 5 7 5 6 1 0 0 1 2 9 9 0 0 2 9 4 0 9 3 9 9 3 3 5 5 9 1 0 5 8 2 0 5 2 8 7 8 9 3 7 2 0 3 3 7 3 9 5 0 3 5 9 2 1 2 8 9 6 2 0 0 1 2 0 1 2 9 0 9 5 9 9 9 2 5 5 3 1 9 5 6 5 9 1 5 0 2 6 3 9 9 0 0
8 9 9 3 0 5 5 9 9 2 1 8 1 9 5 1 6 5 9 8 8 1 1 3 5 1 3 9 5 0 2 1 5 3 4 5 1 6 9 9 9 1 9 1 1 6 4 9 7 3 9 9 5 9 1 5 3 1 7 5 9 5 7 4 7 0 9 7 7 9 8 6 3 4 2 8 9 6 1 9 7 2 9 6 6 9 7 6 8 3 9 0 9 1 3 0 3 7 2 3 7


In [7]:
# テスト文書をファイルに出力 (BoW)
from collections import Counter
with open("lda.test.bow.txt", mode="w") as f:
    print("\n".join([" ".join([f"{k}:{v}" for k, v in sorted(Counter(w).items(), key=lambda x: x[0])]) for w in W]), file=f)

!head -n2 lda.test.bow.txt

0:22 1:12 2:14 3:13 4:1 5:18 6:6 7:8 8:9 9:26
0:5 1:15 2:5 3:11 4:4 5:12 6:8 7:10 8:7 9:24
